# GenerativeAI "Sprachmodell" Projekt

## Verbing mit wandb

In [1]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Adel\_netrc
wandb: Currently logged in as: adel-haj-jumah (adel-haj-jumah-hochschule-hannover) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

1. import datasets

In [11]:
from datasets import load_dataset

ds = load_dataset("wikitext", "wikitext-103-v1", split="train[:5%]")
text = "\n".join(ds['text'])


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

c:\Users\Adel\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Adel\.cache\huggingface\hub\datasets--wikitext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

2. Importieren die notwendigen Bibliotheken

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer
import wandb
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import random

3. Token And Positional Embedding

In [15]:

class TokenAndPositionalEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, max_len):
        super().__init__()
        # ID der Token in einen Vektorraum
        self.token_embed = nn.Embedding(vocab_size, d_model)
        # Positionale Einbettungen (lernen relative Positionen "das ist learnable")
        self.pos_embedding = nn.Parameter(torch.randn(1, 512, d_model)) # batch 1 , bis zu 512 token und vektor größe
    def forward(self, x):
        seq_len = x.size(1) # 1 ist Anzahl der Token"Sequenzlänge" (0 ist batch)
        token_emb = self.token_embed(x)
        pos_emb = self.pos_embedding[:, :seq_len, :]
        return token_emb + pos_emb

4. Masked Multi-Head Self-Attention


In [16]:
class MaskedSelfAttention(nn.Module): # Modell nur auf Wörter schaut, die es schon kennen darf(besucht)
    def __init__(self, d_model, n_heads):
        super().__init__()
        # Ensure the model dimension is divisible by the number of heads
        assert d_model % n_heads == 0
        self.head_dim = d_model // n_heads  # Dimension per attention head
        self.n_heads = n_heads  # Number of attention heads

        # Linear layers for query, key, and value transformations
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        
        # Linear layer for the output transformation
        self.out = nn.Linear(d_model, d_model)